In [2]:
import numpy as np
def are_axes_equivalent(axis2, axis1, angle_threshold=2.0):
    """
    Check if two axes are equivalent considering crystal symmetry and slight deviations by comparing their misorientation angle.
    
    Parameters:
    axis1, axis2 (numpy arrays): The axes to compare
    angle_threshold (float): The angle threshold in degrees for considering the axes equivalent
    
    Returns:
    bool: Whether the axes are equivalent
    """
    # Normalize both axes
    axis1 = axis1 / np.linalg.norm(axis1)
    axis2 = axis2 / np.linalg.norm(axis2)
    
    # Generate all permutations with sign changes of axis1 to account for Cubic symmetry
    axis_permutations = [
        [ axis1[0],  axis1[1],  axis1[2]],
        [-axis1[0],  axis1[1],  axis1[2]],
        [ axis1[0], -axis1[1],  axis1[2]],
        [ axis1[0],  axis1[1], -axis1[2]],
        [-axis1[0], -axis1[1],  axis1[2]],
        [-axis1[0],  axis1[1], -axis1[2]],
        [ axis1[0], -axis1[1], -axis1[2]],
        [-axis1[0], -axis1[1], -axis1[2]],
        [ axis1[1],  axis1[0],  axis1[2]],
        [-axis1[1],  axis1[0],  axis1[2]],
        [ axis1[1], -axis1[0],  axis1[2]],
        [ axis1[1],  axis1[0], -axis1[2]],
        [-axis1[1], -axis1[0],  axis1[2]],
        [-axis1[1],  axis1[0], -axis1[2]],
        [ axis1[1], -axis1[0], -axis1[2]],
        [-axis1[1], -axis1[0], -axis1[2]],
        [ axis1[2],  axis1[1],  axis1[0]],
        [-axis1[2],  axis1[1],  axis1[0]],
        [ axis1[2], -axis1[1],  axis1[0]],
        [ axis1[2],  axis1[1], -axis1[0]],
        [-axis1[2], -axis1[1],  axis1[0]],
        [-axis1[2],  axis1[1], -axis1[0]],
        [ axis1[2], -axis1[1], -axis1[0]],
        [-axis1[2], -axis1[1], -axis1[0]],
        [ axis1[0],  axis1[2],  axis1[1]],
        [-axis1[0],  axis1[2],  axis1[1]],
        [ axis1[0], -axis1[2],  axis1[1]],
        [ axis1[0],  axis1[2], -axis1[1]],
        [-axis1[0], -axis1[2],  axis1[1]],
        [-axis1[0],  axis1[2], -axis1[1]],
        [ axis1[0], -axis1[2], -axis1[1]],
        [-axis1[0], -axis1[2], -axis1[1]],
        [ axis1[1],  axis1[2],  axis1[0]],
        [-axis1[1],  axis1[2],  axis1[0]],
        [ axis1[1], -axis1[2],  axis1[0]],
        [ axis1[1],  axis1[2], -axis1[0]],
        [-axis1[1], -axis1[2],  axis1[0]],
        [-axis1[1],  axis1[2], -axis1[0]],
        [ axis1[1], -axis1[2], -axis1[0]],
        [-axis1[1], -axis1[2], -axis1[0]],
        [ axis1[2],  axis1[0],  axis1[1]],
        [-axis1[2],  axis1[0],  axis1[1]],
        [ axis1[2], -axis1[0],  axis1[1]],
        [ axis1[2],  axis1[0], -axis1[1]],
        [-axis1[2], -axis1[0],  axis1[1]],
        [-axis1[2],  axis1[0], -axis1[1]],
        [ axis1[2], -axis1[0], -axis1[1]],
        [-axis1[2], -axis1[0], -axis1[1]],
    ]
    
    # Check if axis2 is equivalent to any of the permutations of axis1 within the specified angle threshold
    for perm in axis_permutations:
        # Calculate the cosine of the angle between the two vectors
        cos_angle = np.dot(perm, axis2)
        
        # Ensure the value is within the valid range for arccos
        cos_angle = np.clip(cos_angle, -1.0, 1.0)
        
        # Calculate the angle in degrees
        angle = np.degrees(np.arccos(cos_angle))

        
        # Check if the angle is less than the threshold
        if angle <= angle_threshold or (180 - angle) <= angle_threshold:
            print(f"Angle between Axis and CSL Axis {perm:}: {angle:.2f}°")
            return True
    
    return False

def is_sigma_boundary(axis, misorientation_angle, structure_type='FCC', angle_threshold=10.0):
    """
    Determine if the given rotation axis and misorientation angle correspond to a Σ boundary,
    considering the Brandon criterion for deviation and axis symmetry.
    
    Parameters:
    axis (list or tuple): Rotation axis (e.g., [1, 1, 1])
    misorientation_angle (float): Misorientation angle (in degrees)
    structure_type (str): Crystal structure type ('FCC' or 'BCC')
    angle_threshold (float): The angle threshold in degrees for considering the axes equivalent
    
    Returns:
    bool: Whether it is a Σ boundary
    str: Corresponding Σ value (if it is a Σ boundary)
    float: Actual deviation (in degrees)
    """
    # Predefined Σ values with corresponding rotation axes and misorientation angles for both FCC and BCC
    sigma_data = {
        '3': {'FCC': [{'axis': [1, 1, 1], 'angle': 60}],
              'BCC': [{'axis': [1, 1, 1], 'angle': 60}]},
        '5': {'FCC': [{'axis': [1, 0, 0], 'angle': 36.87}],
              'BCC': [{'axis': [1, 0, 0], 'angle': 36.87}]},
        '7': {'FCC': [{'axis': [1, 1, 1], 'angle': 38.21}],
              'BCC': [{'axis': [1, 1, 1], 'angle': 38.21}]},
        '9': {'FCC': [{'axis': [1, 1, 0], 'angle': 38.94}],
              'BCC': [{'axis': [1, 1, 0], 'angle': 38.94}]},
        '11': {'FCC': [{'axis': [1, 1, 0], 'angle': 50.48}],
               'BCC': [{'axis': [1, 1, 0], 'angle': 50.48}]},
        '13a': {'FCC': [{'axis': [1, 0, 0], 'angle': 22.62}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 22.62}]},
        '13b': {'FCC': [{'axis': [1, 1, 1], 'angle': 27.79}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 27.79}]},
        '17a': {'FCC': [{'axis': [1, 0, 0], 'angle': 28.07}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 28.07}]},
        '17b': {'FCC': [{'axis': [2, 2, 1], 'angle': 61.9}],
                'BCC': [{'axis': [2, 2, 1], 'angle': 61.9}]},
        '19a': {'FCC': [{'axis': [1, 1, 0], 'angle': 26.53}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 26.53}]},
        '19b': {'FCC': [{'axis': [1, 1, 1], 'angle': 46.8}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 46.8}]},
        '21a': {'FCC': [{'axis': [1, 1, 1], 'angle': 21.79}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 21.79}]},
        '21b': {'FCC': [{'axis': [2, 1, 1], 'angle': 44.4}],
                'BCC': [{'axis': [2, 1, 1], 'angle': 44.4}]},
        '23': {'FCC': [{'axis': [3, 1, 1], 'angle': 40.5}],
               'BCC': [{'axis': [3, 1, 1], 'angle': 40.5}]},
        '25a': {'FCC': [{'axis': [1, 0, 0], 'angle': 16.3}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 16.3}]},
        '25b': {'FCC': [{'axis': [3, 3, 1], 'angle': 51.7}],
                'BCC': [{'axis': [3, 3, 1], 'angle': 51.7}]},
        '27a': {'FCC': [{'axis': [1, 1, 0], 'angle': 31.59}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 31.59}]},
        '27b': {'FCC': [{'axis': [2, 1, 0], 'angle': 35.43}],
                'BCC': [{'axis': [2, 1, 0], 'angle': 35.43}]},
        '29a': {'FCC': [{'axis': [1, 0, 0], 'angle': 43.6}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 43.6}]},
        '29b': {'FCC': [{'axis': [2, 2, 1], 'angle': 46.4}],
                'BCC': [{'axis': [2, 2, 1], 'angle': 46.4}]},
        '31a': {'FCC': [{'axis': [1, 1, 0], 'angle': 17.9}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 17.9}]},
        '31b': {'FCC': [{'axis': [2, 1, 0], 'angle': 52.2}],
                'BCC': [{'axis': [2, 1, 0], 'angle': 52.2}]},
        '33a': {'FCC': [{'axis': [1, 0, 0], 'angle': 20.0}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 20.0}]},
        '33b': {'FCC': [{'axis': [2, 2, 1], 'angle': 33.6}],
                'BCC': [{'axis': [2, 2, 1], 'angle': 33.6}]},
        '33c': {'FCC': [{'axis': [1, 1, 0], 'angle': 59.0}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 59.0}]},
        '35a': {'FCC': [{'axis': [2, 1, 0], 'angle': 34.0}],
                'BCC': [{'axis': [2, 1, 0], 'angle': 34.0}]},
        '35b': {'FCC': [{'axis': [1, 0, 0], 'angle': 43.2}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 43.2}]},
        '37a': {'FCC': [{'axis': [2, 2, 1], 'angle': 18.9}],
                'BCC': [{'axis': [2, 2, 1], 'angle': 18.9}]},
        '37b': {'FCC': [{'axis': [1, 1, 0], 'angle': 43.1}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 43.1}]},
        '37c': {'FCC': [{'axis': [2, 1, 0], 'angle': 50.6}],
                'BCC': [{'axis': [2, 1, 0], 'angle': 50.6}]},
        '39a': {'FCC': [{'axis': [1, 0, 0], 'angle': 32.2}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 32.2}]},
        '39b': {'FCC': [{'axis': [2, 2, 1], 'angle': 50.1}],
                'BCC': [{'axis': [2, 2, 1], 'angle': 50.1}]},
        '41a': {'FCC': [{'axis': [1, 1, 0], 'angle': 12.7}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 12.7}]},
        '41b': {'FCC': [{'axis': [2, 1, 0], 'angle': 40.9}],
                'BCC': [{'axis': [2, 1, 0], 'angle': 40.9}]},
        '41c': {'FCC': [{'axis': [1, 0, 0], 'angle': 55.9}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 55.9}]},
        '43a': {'FCC': [{'axis': [2, 2, 1], 'angle': 15.2}],
                'BCC': [{'axis': [2, 2, 1], 'angle': 15.2}]},
        '43b': {'FCC': [{'axis': [1, 1, 0], 'angle': 27.9}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 27.9}]},
        '43c': {'FCC': [{'axis': [2, 1, 0], 'angle': 60.8}],
                'BCC': [{'axis': [2, 1, 0], 'angle': 60.8}]},
        '45a': {'FCC': [{'axis': [1, 0, 0], 'angle': 28.6}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 28.6}]},
        '45b': {'FCC': [{'axis': [2, 2, 1], 'angle': 36.9}],
                'BCC': [{'axis': [2, 2, 1], 'angle': 36.9}]},
        '45c': {'FCC': [{'axis': [1, 1, 0], 'angle': 53.1}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 53.1}]},
        '47a': {'FCC': [{'axis': [2, 1, 0], 'angle': 37.1}],
                'BCC': [{'axis': [2, 1, 0], 'angle': 37.1}]},
        '47b': {'FCC': [{'axis': [1, 0, 0], 'angle': 43.7}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 43.7}]},
        '49a': {'FCC': [{'axis': [2, 2, 1], 'angle': 43.6}],
                'BCC': [{'axis': [2, 2, 1], 'angle': 43.6}]},
        '49b': {'FCC': [{'axis': [1, 1, 0], 'angle': 43.6}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 43.6}]},
        '49c': {'FCC': [{'axis': [2, 1, 0], 'angle': 49.2}],
                'BCC': [{'axis': [2, 1, 0], 'angle': 49.2}]}
    }
    
    # Normalize the given rotation axis
    axis = np.array(axis)
    
    for sigma, configs in sigma_data.items():
        if structure_type not in configs:
            continue
        for config in configs[structure_type]:
            predefined_axis = np.array(config['axis'])
            
            # Check if the given axis is equivalent to the predefined axis
            if are_axes_equivalent(axis, predefined_axis, angle_threshold):
                predefined_angle = config['angle']
                
                # Calculate the deviation
                deviation = abs(misorientation_angle - predefined_angle)
                
                # Calculate the Brandon criterion threshold
                brandon_threshold = 15 / np.sqrt(float(sigma.rstrip('ab')))
                
                # Check if the misorientation angle matches within the Brandon criterion threshold
                if deviation <= brandon_threshold:
                    return True, sigma, deviation
    
    return False, None, None

# Example usage
axis1 = [17, 17, -16]
misorientation_angle1 = 59.6
structure_type = 'FCC'  # 'FCC' or 'BCC'

is_sigma1, sigma_value1, deviation1 = is_sigma_boundary(axis1, misorientation_angle1, structure_type)
if is_sigma1:
    print(f"This is a Σ{sigma_value1} boundary with a deviation of {deviation1:.2f} degrees")
else:
    print("This is not a Σ boundary")

axis2 = [16, 16, -16]
misorientation_angle2 = 60

is_sigma2, sigma_value2, deviation2 = is_sigma_boundary(axis2, misorientation_angle2, structure_type)
if is_sigma2:
    print(f"This is a Σ{sigma_value2} boundary with a deviation of {deviation2:.2f} degrees")
else:
    print("This is not a Σ boundary")

    
axis3 = [0.1142, 0.0523, 0.9921]
misorientation_angle3 = 19.24199995
is_sigma3, sigma_value3, deviation3 = is_sigma_boundary(axis3, misorientation_angle3, structure_type)
if is_sigma3:
    print(f"This is a Σ{sigma_value3} boundary with a deviation of {deviation3:.2f} degrees")
else:
    print("This is not a Σ boundary")
    


Angle between Axis and CSL Axis [0.5773502691896258, 0.5773502691896258, -0.5773502691896258]: 1.62°
This is a Σ3 boundary with a deviation of 0.40 degrees
Angle between Axis and CSL Axis [0.5773502691896258, 0.5773502691896258, -0.5773502691896258]: 0.00°
This is a Σ3 boundary with a deviation of 0.00 degrees
Angle between Axis and CSL Axis [0.0, 0.0, 1.0]: 7.22°
Angle between Axis and CSL Axis [0.0, 0.0, 1.0]: 7.22°
This is a Σ13a boundary with a deviation of 3.38 degrees


In [5]:
import numpy as np

def are_axes_equivalent(axis2, axis1, angle_threshold=2.0):
    """
    Check if two axes are equivalent considering crystal symmetry and slight deviations by comparing their misorientation angle.
    
    Parameters:
    axis1, axis2 (numpy arrays): The axes to compare
    angle_threshold (float): The angle threshold in degrees for considering the axes equivalent
    
    Returns:
    bool: Whether the axes are equivalent
    """
    # Normalize both axes
    axis1 = axis1 / np.linalg.norm(axis1)
    axis2 = axis2 / np.linalg.norm(axis2)
    
    # Generate all permutations with sign changes of axis1 to account for symmetry of Cubic system
    axis_permutations = [
        [ axis1[0],  axis1[1],  axis1[2]],
        [-axis1[0],  axis1[1],  axis1[2]],
        [ axis1[0], -axis1[1],  axis1[2]],
        [ axis1[0],  axis1[1], -axis1[2]],
        [-axis1[0], -axis1[1],  axis1[2]],
        [-axis1[0],  axis1[1], -axis1[2]],
        [ axis1[0], -axis1[1], -axis1[2]],
        [-axis1[0], -axis1[1], -axis1[2]],
        [ axis1[1],  axis1[0],  axis1[2]],
        [-axis1[1],  axis1[0],  axis1[2]],
        [ axis1[1], -axis1[0],  axis1[2]],
        [ axis1[1],  axis1[0], -axis1[2]],
        [-axis1[1], -axis1[0],  axis1[2]],
        [-axis1[1],  axis1[0], -axis1[2]],
        [ axis1[1], -axis1[0], -axis1[2]],
        [-axis1[1], -axis1[0], -axis1[2]],
        [ axis1[2],  axis1[1],  axis1[0]],
        [-axis1[2],  axis1[1],  axis1[0]],
        [ axis1[2], -axis1[1],  axis1[0]],
        [ axis1[2],  axis1[1], -axis1[0]],
        [-axis1[2], -axis1[1],  axis1[0]],
        [-axis1[2],  axis1[1], -axis1[0]],
        [ axis1[2], -axis1[1], -axis1[0]],
        [-axis1[2], -axis1[1], -axis1[0]],
        [ axis1[0],  axis1[2],  axis1[1]],
        [-axis1[0],  axis1[2],  axis1[1]],
        [ axis1[0], -axis1[2],  axis1[1]],
        [ axis1[0],  axis1[2], -axis1[1]],
        [-axis1[0], -axis1[2],  axis1[1]],
        [-axis1[0],  axis1[2], -axis1[1]],
        [ axis1[0], -axis1[2], -axis1[1]],
        [-axis1[0], -axis1[2], -axis1[1]],
        [ axis1[1],  axis1[2],  axis1[0]],
        [-axis1[1],  axis1[2],  axis1[0]],
        [ axis1[1], -axis1[2],  axis1[0]],
        [ axis1[1],  axis1[2], -axis1[0]],
        [-axis1[1], -axis1[2],  axis1[0]],
        [-axis1[1],  axis1[2], -axis1[0]],
        [ axis1[1], -axis1[2], -axis1[0]],
        [-axis1[1], -axis1[2], -axis1[0]],
        [ axis1[2],  axis1[0],  axis1[1]],
        [-axis1[2],  axis1[0],  axis1[1]],
        [ axis1[2], -axis1[0],  axis1[1]],
        [ axis1[2],  axis1[0], -axis1[1]],
        [-axis1[2], -axis1[0],  axis1[1]],
        [-axis1[2],  axis1[0], -axis1[1]],
        [ axis1[2], -axis1[0], -axis1[1]],
        [-axis1[2], -axis1[0], -axis1[1]],
    ]
    
    
    # Check if axis2 is equivalent to any of the permutations of axis1 within the specified angle threshold
    for perm in axis_permutations:
        # Calculate the cosine of the angle between the two vectors
        cos_angle = np.dot(perm, axis2)
        
        # Ensure the value is within the valid range for arccos
        cos_angle = np.clip(cos_angle, -1.0, 1.0)
        
        # Calculate the angle in degrees
        angle = np.degrees(np.arccos(cos_angle))

        
        # Check if the angle is less than the threshold
        if angle <= angle_threshold or (180 - angle) <= angle_threshold:
            #print(f"Angle between Axis and CSL Axis {perm}: {angle:.2f}°")
            return True
    
    return False

def is_sigma_boundary(axis, misorientation_angle, structure_type='FCC', angle_threshold=10.0):
    """
    Determine if the given rotation axis and misorientation angle correspond to a Σ boundary,
    considering the Brandon criterion for deviation and axis symmetry.
    
    Parameters:
    axis (list or tuple): Rotation axis (e.g., [1, 1, 1])
    misorientation_angle (float): Misorientation angle (in degrees)
    structure_type (str): Crystal structure type ('FCC' or 'BCC')
    angle_threshold (float): The angle threshold in degrees for considering the axes equivalent
    
    Returns:
    bool: Whether it is a Σ boundary
    str: Corresponding Σ value (if it is a Σ boundary)
    float: Actual deviation (in degrees)
    """
    # Predefined Σ values with corresponding rotation axes and misorientation angles for both FCC and BCC
    sigma_data = {
        '3': {'FCC': [{'axis': [1, 1, 1], 'angle': 60}],
              'BCC': [{'axis': [1, 1, 1], 'angle': 60}]},
        '5': {'FCC': [{'axis': [1, 0, 0], 'angle': 36.87}],
              'BCC': [{'axis': [1, 0, 0], 'angle': 36.87}]},
        '7': {'FCC': [{'axis': [1, 1, 1], 'angle': 38.21}],
              'BCC': [{'axis': [1, 1, 1], 'angle': 38.21}]},
        '9': {'FCC': [{'axis': [1, 1, 0], 'angle': 38.94}],
              'BCC': [{'axis': [1, 1, 0], 'angle': 38.94}]},
        '11': {'FCC': [{'axis': [1, 1, 0], 'angle': 50.48}],
               'BCC': [{'axis': [1, 1, 0], 'angle': 50.48}]},
        '13a': {'FCC': [{'axis': [1, 0, 0], 'angle': 22.62}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 22.62}]},
        '13b': {'FCC': [{'axis': [1, 1, 1], 'angle': 27.79}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 27.79}]},
        '17a': {'FCC': [{'axis': [1, 0, 0], 'angle': 28.07}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 28.07}]},
        '17b': {'FCC': [{'axis': [2, 2, 1], 'angle': 61.9}],
                'BCC': [{'axis': [2, 2, 1], 'angle': 61.9}]},
        '19a': {'FCC': [{'axis': [1, 1, 0], 'angle': 26.53}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 26.53}]},
        '19b': {'FCC': [{'axis': [1, 1, 1], 'angle': 46.8}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 46.8}]},
        '21a': {'FCC': [{'axis': [1, 1, 1], 'angle': 21.79}],
                'BCC': [{'axis': [1, 1, 1], 'angle': 21.79}]},
        '21b': {'FCC': [{'axis': [2, 1, 1], 'angle': 44.4}],
                'BCC': [{'axis': [2, 1, 1], 'angle': 44.4}]},
        '23': {'FCC': [{'axis': [3, 1, 1], 'angle': 40.5}],
               'BCC': [{'axis': [3, 1, 1], 'angle': 40.5}]},
        '25a': {'FCC': [{'axis': [1, 0, 0], 'angle': 16.3}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 16.3}]},
        '25b': {'FCC': [{'axis': [3, 3, 1], 'angle': 51.7}],
                'BCC': [{'axis': [3, 3, 1], 'angle': 51.7}]},
        '27a': {'FCC': [{'axis': [1, 1, 0], 'angle': 31.59}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 31.59}]},
        '27b': {'FCC': [{'axis': [2, 1, 0], 'angle': 35.43}],
                'BCC': [{'axis': [2, 1, 0], 'angle': 35.43}]},
        '29a': {'FCC': [{'axis': [1, 0, 0], 'angle': 43.6}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 43.6}]},
        '29b': {'FCC': [{'axis': [2, 2, 1], 'angle': 46.4}],
                'BCC': [{'axis': [2, 2, 1], 'angle': 46.4}]},
        '31a': {'FCC': [{'axis': [1, 1, 0], 'angle': 17.9}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 17.9}]},
        '31b': {'FCC': [{'axis': [2, 1, 0], 'angle': 52.2}],
                'BCC': [{'axis': [2, 1, 0], 'angle': 52.2}]},
        '33a': {'FCC': [{'axis': [1, 0, 0], 'angle': 20.0}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 20.0}]},
        '33b': {'FCC': [{'axis': [2, 2, 1], 'angle': 33.6}],
                'BCC': [{'axis': [2, 2, 1], 'angle': 33.6}]},
        '33c': {'FCC': [{'axis': [1, 1, 0], 'angle': 59.0}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 59.0}]},
        '35a': {'FCC': [{'axis': [2, 1, 0], 'angle': 34.0}],
                'BCC': [{'axis': [2, 1, 0], 'angle': 34.0}]},
        '35b': {'FCC': [{'axis': [1, 0, 0], 'angle': 43.2}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 43.2}]},
        '37a': {'FCC': [{'axis': [2, 2, 1], 'angle': 18.9}],
                'BCC': [{'axis': [2, 2, 1], 'angle': 18.9}]},
        '37b': {'FCC': [{'axis': [1, 1, 0], 'angle': 43.1}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 43.1}]},
        '37c': {'FCC': [{'axis': [2, 1, 0], 'angle': 50.6}],
                'BCC': [{'axis': [2, 1, 0], 'angle': 50.6}]},
        '39a': {'FCC': [{'axis': [1, 0, 0], 'angle': 32.2}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 32.2}]},
        '39b': {'FCC': [{'axis': [2, 2, 1], 'angle': 50.1}],
                'BCC': [{'axis': [2, 2, 1], 'angle': 50.1}]},
        '41a': {'FCC': [{'axis': [1, 1, 0], 'angle': 12.7}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 12.7}]},
        '41b': {'FCC': [{'axis': [2, 1, 0], 'angle': 40.9}],
                'BCC': [{'axis': [2, 1, 0], 'angle': 40.9}]},
        '41c': {'FCC': [{'axis': [1, 0, 0], 'angle': 55.9}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 55.9}]},
        '43a': {'FCC': [{'axis': [2, 2, 1], 'angle': 15.2}],
                'BCC': [{'axis': [2, 2, 1], 'angle': 15.2}]},
        '43b': {'FCC': [{'axis': [1, 1, 0], 'angle': 27.9}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 27.9}]},
        '43c': {'FCC': [{'axis': [2, 1, 0], 'angle': 60.8}],
                'BCC': [{'axis': [2, 1, 0], 'angle': 60.8}]},
        '45a': {'FCC': [{'axis': [1, 0, 0], 'angle': 28.6}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 28.6}]},
        '45b': {'FCC': [{'axis': [2, 2, 1], 'angle': 36.9}],
                'BCC': [{'axis': [2, 2, 1], 'angle': 36.9}]},
        '45c': {'FCC': [{'axis': [1, 1, 0], 'angle': 53.1}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 53.1}]},
        '47a': {'FCC': [{'axis': [2, 1, 0], 'angle': 37.1}],
                'BCC': [{'axis': [2, 1, 0], 'angle': 37.1}]},
        '47b': {'FCC': [{'axis': [1, 0, 0], 'angle': 43.7}],
                'BCC': [{'axis': [1, 0, 0], 'angle': 43.7}]},
        '49a': {'FCC': [{'axis': [2, 2, 1], 'angle': 43.6}],
                'BCC': [{'axis': [2, 2, 1], 'angle': 43.6}]},
        '49b': {'FCC': [{'axis': [1, 1, 0], 'angle': 43.6}],
                'BCC': [{'axis': [1, 1, 0], 'angle': 43.6}]},
        '49c': {'FCC': [{'axis': [2, 1, 0], 'angle': 49.2}],
                'BCC': [{'axis': [2, 1, 0], 'angle': 49.2}]}
    }
    
    # Normalize the given rotation axis
    axis = np.array(axis)
    
    for sigma, configs in sigma_data.items():
        if structure_type not in configs:
            continue
        for config in configs[structure_type]:
            predefined_axis = np.array(config['axis'])
            
            # Check if the given axis is equivalent to the predefined axis
            if are_axes_equivalent(axis, predefined_axis, angle_threshold):
                predefined_angle = config['angle']
                
                # Calculate the deviation
                deviation = abs(misorientation_angle - predefined_angle)
                
                # Calculate the Brandon criterion threshold
                brandon_threshold = 15 / np.sqrt(float(sigma.rstrip('abc')))
                
                # Check if the misorientation angle matches within the Brandon criterion threshold
                if deviation <= brandon_threshold:
                    return True, sigma, deviation
    
    return False, None, None

# Given data
misori_angle = [39.99032809, 34.61751482, 31.8568031, 33.75154261, 47.8394252, 52.18859909, 59.6111677, 45.55147369, 19.51118361, 34.19565999, 28.66987262, 52.3075686, 29.70978892]

Axis = np.array([
    [0.2875, 0.6517, 0.7019],
    [0.4314, 0.0756, 0.899],
    [0.7027, 0.0415, 0.7103],
    [0.1249, 0.0627, 0.9902],
    [0.2368, 0.4618, 0.8548],
    [0.2609, 0.4685, 0.8441],
    [0.5721, 0.5795, 0.5804],
    [0.1318, 0.5213, 0.8432],
    [0.3954, 0.2071, 0.8948],
    [0.0288, 0.1988, 0.9796],
    [0.39, 0.3213, 0.8629],
    [0.6881, 0.1837, 0.702],
    [0.6183, 0.4417, 0.65],
])

# Check each axis and misorientation angle
results = []
for axis, angle in zip(Axis, misori_angle):
    is_sigma, sigma_value, deviation = is_sigma_boundary(axis, angle, structure_type='BCC')
    if is_sigma:
        results.append((axis, angle, sigma_value, deviation))
        print(f"Axis: {axis}, Misorientation Angle: {angle:.2f}°, Σ Boundary: {sigma_value}, Deviation: {deviation:.2f}°")
    else:
        results.append((axis, angle, None, None))
        print(f"Axis: {axis}, Misorientation Angle: {angle:.2f}°, Not a Σ Boundary")



Axis: [0.2875 0.6517 0.7019], Misorientation Angle: 39.99°, Not a Σ Boundary
Axis: [0.4314 0.0756 0.899 ], Misorientation Angle: 34.62°, Σ Boundary: 27b, Deviation: 0.81°
Axis: [0.7027 0.0415 0.7103], Misorientation Angle: 31.86°, Σ Boundary: 27a, Deviation: 0.27°
Axis: [0.1249 0.0627 0.9902], Misorientation Angle: 33.75°, Σ Boundary: 5, Deviation: 3.12°
Axis: [0.2368 0.4618 0.8548], Misorientation Angle: 47.84°, Not a Σ Boundary
Axis: [0.2609 0.4685 0.8441], Misorientation Angle: 52.19°, Not a Σ Boundary
Axis: [0.5721 0.5795 0.5804], Misorientation Angle: 59.61°, Σ Boundary: 3, Deviation: 0.39°
Axis: [0.1318 0.5213 0.8432], Misorientation Angle: 45.55°, Not a Σ Boundary
Axis: [0.3954 0.2071 0.8948], Misorientation Angle: 19.51°, Not a Σ Boundary
Axis: [0.0288 0.1988 0.9796], Misorientation Angle: 34.20°, Not a Σ Boundary
Axis: [0.39   0.3213 0.8629], Misorientation Angle: 28.67°, Not a Σ Boundary
Axis: [0.6881 0.1837 0.702 ], Misorientation Angle: 52.31°, Σ Boundary: 25b, Deviation: 0